In [8]:
import pandas as pd
import difflib 
import re 

# --- 1. Definición de archivos y columnas ---
archivo_maestro = r"..\data\raw\hermosillo_colonias_poblacion.csv"
columna_maestra = "nom_col" # Columna con nombres correctos

archivo_a_corregir = r"..\data\raw\213.csv"
columna_a_corregir = "COLONIA" # Columna con nombres "sucios"

archivo_salida = r"..\data\raw\colonias_corregidas.csv"

# --- 2. Configuración y Función de Normalización ---

SIMILARITY_THRESHOLD = 0.9

def normalize_name(name):
    """
    Limpia un nombre de colonia usando regex para una mejor comparación.
    """
    if not isinstance(name, str):
        return ""
    
    name = name.upper()
    name = name.replace('Á', 'A').replace('É', 'E').replace('Í', 'I').replace('Ó', 'O').replace('Ú', 'U')
    
    prefix_pattern = r'^(FRACCIONAMIENTO|FRACC|COLONIA|COL|RESIDENCIAL|RES|AMPLIACION|AMP|INVASION|INV|PRIVADA|PRIV)\.?\s+'
    name = re.sub(prefix_pattern, '', name, flags=re.IGNORECASE)
    
    name = re.sub(r'[^\w\s]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    
    return name

# --- 3. Carga de Datos ---
try:
    df_poblacion = pd.read_csv(archivo_maestro)
    df_crimenes = pd.read_csv(archivo_a_corregir)
    print("Archivos cargados exitosamente.")
except Exception as e:
    print(f"Error al cargar los archivos: {e}")
    exit()

# --- 4. Preparación de Listas de Colonias (Lógica Modificada) ---

try:
    clean_colonias_originales = df_poblacion[columna_maestra].dropna().astype(str).unique()
    print(f"Se encontraron {len(clean_colonias_originales)} colonias únicas en el archivo maestro.")
except KeyError:
    print(f"Error: No se encontró la columna '{columna_maestra}' en '{archivo_maestro}'.")
    exit()

print("Normalizando lista maestra con Regex...")
norm_to_clean_map = {}
for clean_name in clean_colonias_originales:
    norm_name = normalize_name(clean_name)
    if norm_name and norm_name not in norm_to_clean_map:
        norm_to_clean_map[norm_name] = clean_name

normalized_clean_list = list(norm_to_clean_map.keys())
print(f"Se generaron {len(normalized_clean_list)} nombres normalizados únicos para el mapeo.")

try:
    dirty_colonias_list = df_crimenes[columna_a_corregir].dropna().astype(str).str.upper().unique()
    print(f"Se encontraron {len(dirty_colonias_list)} colonias únicas en el archivo a corregir.")
except KeyError:
    print(f"Error: No se encontró la columna '{columna_a_corregir}' en '{archivo_a_corregir}'.")
    exit()


# --- 5. Lógica de Corrección (Mapeo Modificado) ---
print(f"\nIniciando mapeo de corrección. Umbral de similitud = {SIMILARITY_THRESHOLD}...")

correction_map = {}
no_match_count = 0

for dirty_name in dirty_colonias_list: 
    normalized_dirty = normalize_name(dirty_name)
    
    if not normalized_dirty: 
        correction_map[dirty_name] = dirty_name 
        no_match_count += 1
        continue

    matches = difflib.get_close_matches(
        normalized_dirty,
        normalized_clean_list, 
        n=1,
        cutoff=SIMILARITY_THRESHOLD
    )

    if matches:
        best_normalized_match = matches[0] 
        best_clean_match = norm_to_clean_map[best_normalized_match]
        correction_map[dirty_name] = best_clean_match
    else:
        correction_map[dirty_name] = dirty_name
        no_match_count += 1

print(f"Se corregirán {len(dirty_colonias_list) - no_match_count} nombres de colonias.")
print(f"Se mantendrán {no_match_count} nombres de colonias sin cambios.")

# --- 6. Aplicación del Mapa y Guardado ---

print("\nAplicando correcciones al DataFrame de crímenes...")

dirty_names_uppercase = df_crimenes[columna_a_corregir].astype(str).str.upper()
df_crimenes['COLONIA_CORREGIDA'] = dirty_names_uppercase.map(correction_map)
df_crimenes['COLONIA_CORREGIDA'].fillna(df_crimenes[columna_a_corregir], inplace=True)

try:
    df_crimenes.to_csv(archivo_salida, index=False, encoding='utf-8-sig')
    print(f"\n¡Proceso completado! Se ha guardado el archivo corregido como: '{archivo_salida}'")
except Exception as e:
    print(f"\nError al guardar el archivo: {e}")


# --- 7. (NUEVO) Guardar el Mapeo de Corrección ---
print("\nGuardando el diccionario de mapeo en un CSV...")

# Define el nombre del archivo de mapeo
archivo_mapeo_salida = r"..\data\raw\mapeo_correccion_colonias.csv"

try:
    # Convierte el diccionario 'correction_map' a un DataFrame
    df_mapeo = pd.DataFrame(
        list(correction_map.items()), 
        columns=['COLONIA_ORIGINAL_SUCIA', 'COLONIA_CORREGIDA_MAESTRA']
    )
    
    # Ordena para que sea más fácil de revisar
    df_mapeo = df_mapeo.sort_values(by=['COLONIA_CORREGIDA_MAESTRA', 'COLONIA_ORIGINAL_SUCIA'])
    
    # Guarda el DataFrame de mapeo en un CSV
    df_mapeo.to_csv(archivo_mapeo_salida, index=False, encoding='utf-8-sig')
    
    print(f"¡Éxito! Mapeo guardado en: '{archivo_mapeo_salida}'")

except Exception as e:
    print(f"\nError al guardar el archivo de mapeo: {e}")

Archivos cargados exitosamente.
Se encontraron 660 colonias únicas en el archivo maestro.
Normalizando lista maestra con Regex...
Se generaron 655 nombres normalizados únicos para el mapeo.
Se encontraron 1407 colonias únicas en el archivo a corregir.

Iniciando mapeo de corrección. Umbral de similitud = 0.9...
Se corregirán 622 nombres de colonias.
Se mantendrán 785 nombres de colonias sin cambios.

Aplicando correcciones al DataFrame de crímenes...


C:\Users\joel_\AppData\Local\Temp\ipykernel_28008\2441929760.py:110: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_crimenes['COLONIA_CORREGIDA'].fillna(df_crimenes[columna_a_corregir], inplace=True)



¡Proceso completado! Se ha guardado el archivo corregido como: '..\data\raw\colonias_corregidas.csv'

Guardando el diccionario de mapeo en un CSV...
¡Éxito! Mapeo guardado en: '..\data\raw\mapeo_correccion_colonias.csv'


In [9]:
df_crimenes.head(20)

,COLONIA,TIPO DE INCIDENTE,FECHA,HORA,COLONIA_CORREGIDA
0,10 DE MAYO,PORTACIÓN DE ARMAS O CARTUCHOS,2018-07-30,22,10 DE MAYO
1,10 DE MAYO,PERSONA AGRESIVA,2018-11-16,7,10 DE MAYO
2,10 DE MAYO,APOYO A LA CIUDADANÍA,2018-11-16,9,10 DE MAYO
3,14 DE MARZO,ALLANAMIENTO DE MORADA,2018-01-29,4,4 DE MARZO
4,1RO DE JULIO,VEHÍCULO A EXCESO DE VELOCIDAD,2018-03-12,3,1RO DE JULIO
5,22 DE SEPTIEMBRE,RUIDO EXCESIVO,2018-01-01,19,22 DE SEPTIEMBRE
6,22 DE SEPTIEMBRE,RUIDO EXCESIVO,2018-01-01,19,22 DE SEPTIEMBRE
7,22 DE SEPTIEMBRE,ALTERACIÓN DEL ORDEN PÚBLICO POR PERSONA DROGADA,2018-01-01,23,22 DE SEPTIEMBRE
8,22 DE SEPTIEMBRE,VIOLENCIA FAMILIAR,2018-01-07,2,22 DE SEPTIEMBRE
9,22 DE SEPTIEMBRE,OTROS TIPOS DE ALTERACIÓN AL ORDEN PÚBLICO,2018-01-07,3,22 DE SEPTIEMBRE


In [10]:
#sacar colonias unicas de df_crimenes['COLONIA_CORREGIDA']
unique_colonias = df_crimenes['COLONIA_CORREGIDA'].dropna().unique()
len(unique_colonias)

1253